In [0]:
!pip install -q yahoo_fin

### Importing Required Libraries

In [0]:
import requests
import http
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

import pandas as pd
from yahoo_fin import stock_info
from pandas_datareader import DataReader
import numpy as np
import urllib3
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import os
import time
import matplotlib.pyplot as plt
%matplotlib inline

from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

### Retrieving S&P 500 stock information

In [0]:
#Function to retrieve S&P 500 recommendations from Yahoo Finance
def get_recommendations(tickers):
    recommendations = []
    
    for ticker in tickers:
  
        #Yahoo Finance URL for current stock ticker
        url = 'https://query2.finance.yahoo.com/v10/finance/quoteSummary/'
        url+= ticker
        url += '?formatted=true&crumb=swg7qs5y9UP&lang=en-US&region=US&' \
                'modules=upgradeDowngradeHistory,recommendationTrend,' \
                'financialData,earningsHistory,earningsTrend,industryTrend&' \
                'corsDomain=finance.yahoo.com'
                  
        data = requests.get(url)
        if(not data.ok):
            recommendataion = -1
        
        #Parsing JSON response to retrieve recommendation rating
        try:
          result = data.json()['quoteSummary']['result'][0]
          recommendation = result['financialData']['recommendationMean']['fmt']
        except:
          recommendation = -1

        recommendations.append(recommendation)
    
    return recommendations


#S&P 500 Tickers
tickers = stock_info.tickers_sp500()
recommendations = get_recommendations(tickers)

In [5]:
#Creating a dataframe with stocks and their respective recommendations
columns = ['Company','Recommendations']
data = pd.DataFrame(list(zip(tickers,recommendations)), columns = columns)

#Converting Recommendations column to float
data['Recommendations'] = pd.to_numeric(data['Recommendations'])

data = data[data.Recommendations != -1]

data.sort_values(by=['Recommendations'],)

,Company,Recommendations
387,PWR,1.5
284,LKQ,1.5
72,BSX,1.6
150,DXCM,1.6
255,J,1.6
...,...,...
1,AAL,3.3
154,ED,3.4
61,BEN,3.4
309,MKC,3.5


In [69]:
#Creating Seperate DataFrames to house hold, sell and buy stocka
hold = data[data.Recommendations == 3]
buy = data[data.Recommendations <= 2]
sell = data[data.Recommendations >= 4]

#Combining theses data frames into one big DF
data_lst = [buy,hold,sell]
new_data = pd.concat(data_lst)
new_data.reset_index(level = 0, inplace = True)

new_data

,index,Company,Recommendations
0,3,AAPL,2.0
1,7,ABT,1.8
2,9,ADBE,1.9
3,10,ADI,2.0
4,15,AEE,2.0
...,...,...,...
128,279,LEG,3.0
129,401,ROK,3.0
130,418,SO,3.0
131,452,UA,3.0


In [0]:
#Function is used to populate a List with Recommended Stocks from Yahoo Finance
def get_tickers(data):
  tickers = []

  for index, row in data.iterrows():
    tickers.append(row.Company)

  return tickers

#Function is used to fetch news information of stocks recomended by Yahoo Finance
def fetch_news(tickers):
  news = {}

  for ticker in tickers:
    url = 'https://finviz.com/quote.ashx?t=' + ticker

    #Scraping FinWiz webpage to get news involving current stock
    req = requests.get(url=url, headers={'User-Agent':'my-app/0.0.1'}) 
    response = req.content
  

    html_page = BeautifulSoup(response)
    news_table = html_page.find(id='news-table')
    
    news[ticker] = news_table

  return news

tickers = get_tickers(new_data)

news = fetch_news(tickers)

In [71]:
#Function is used to parse news info and create a dataframe involving each company and relevant news
def news_data(news):
  news_table = []

  for file_name, curr_news in news.items():
    for each in curr_news.findAll('tr'):
      
      #Scraping News Table to retrieve relevant news and timestamp
      text = each.a.get_text()
      data = each.td.text.split()

      if(len(data) == 1):
        time = data[0]
      else:
        date = data[0]
        time = data[1]
      
      ticker = file_name.split('_')[0]

      ticker_info = [ticker,date,time,text]
      news_table.append(ticker_info)

  #Creating of a DataFrame with ticker and news information
  columns = ['Ticker','Date','Time','Headline']
  parsed_news = pd.DataFrame(news_table,columns=columns)

  parsed_news['Date'] = pd.to_datetime(parsed_news.Date).dt.date

  return parsed_news

parsed_news = news_data(news)

#Grouping the headlines for each comapny into one string
parsed_news = parsed_news.groupby(['Ticker'], as_index = False).agg({'Headline':''.join}, Inplace = True)
parsed_news

,Ticker,Headline
0,ALLE,Why Is Allegion (ALLE) Down 0.9% Since Last Ea...
1,CLX,Should We Waste This Crisis?Here's what boomin...
2,CTSH,8 Stocks Diamond Hill Capital Continues to Buy...
3,DISCK,"John Paulson Adds 2 Stocks to Portfolio, Boost..."
4,JKHY,UNIFY Financial Credit Union Moving its Symita...
5,KIM,Here's Why You Should Hold On to Kimco Realty ...
6,KSS,A Bad Earnings Report for Nordstrom Is Good En...
7,L,Dow futures gain as investors watch for earnin...
8,LEG,Edited Transcript of LEG earnings conference c...
9,LKQ,"Jeff Ubben's ValueAct Sells FedEx, Buys 2 New ..."


### Sentiment Analysis

In [52]:
analyzer = SentimentIntensityAnalyzer()

#Applying the Sentiment Analyzer to the headline column
scores = parsed_news['Headline'].apply(analyzer.polarity_scores).tolist()

scores_df = pd.DataFrame(scores)

#Joining the scores DataFrame with the Ticker DataFrame
parsed_news['compound'] = scores_df['compound']

#Renaming Ticker Column to prepare for a Natural Join
parsed_news = parsed_news.rename(columns={'Ticker':'Company'})
parsed_news

,Company,Headline,compound
0,ALLE,Why Is Allegion (ALLE) Down 0.9% Since Last Ea...,0.9992
1,CLX,Should We Waste This Crisis?Here's what boomin...,0.9914
2,CTSH,8 Stocks Diamond Hill Capital Continues to Buy...,0.9909
3,DISCK,"John Paulson Adds 2 Stocks to Portfolio, Boost...",0.9802
4,JKHY,UNIFY Financial Credit Union Moving its Symita...,0.9983
5,KIM,Here's Why You Should Hold On to Kimco Realty ...,0.9989
6,KSS,A Bad Earnings Report for Nordstrom Is Good En...,-0.9871
7,L,Dow futures gain as investors watch for earnin...,0.9770
8,LEG,Edited Transcript of LEG earnings conference c...,0.9785
9,LKQ,"Jeff Ubben's ValueAct Sells FedEx, Buys 2 New ...",0.9849


In [54]:
#Merging Yahoo Finance Recommendations with Sentiment Score
final_data = pd.merge(parsed_news,new_data, on='Company')
del final_data['index']

final_data

,Company,Headline,compound,Recommendations
0,ALLE,Why Is Allegion (ALLE) Down 0.9% Since Last Ea...,0.9992,3.0
1,CLX,Should We Waste This Crisis?Here's what boomin...,0.9914,3.0
2,CTSH,8 Stocks Diamond Hill Capital Continues to Buy...,0.9909,3.0
3,DISCK,"John Paulson Adds 2 Stocks to Portfolio, Boost...",0.9802,3.0
4,JKHY,UNIFY Financial Credit Union Moving its Symita...,0.9983,3.0
5,KIM,Here's Why You Should Hold On to Kimco Realty ...,0.9989,3.0
6,KSS,A Bad Earnings Report for Nordstrom Is Good En...,-0.9871,3.0
7,L,Dow futures gain as investors watch for earnin...,0.9770,3.0
8,LEG,Edited Transcript of LEG earnings conference c...,0.9785,3.0
9,LKQ,"Jeff Ubben's ValueAct Sells FedEx, Buys 2 New ...",0.9849,1.5


In [68]:
buy_stocks = []
sell_stocks = []
hold_stocks = []

for index, row in final_data.iterrows():
  curr_stock = (row.Company, row.compound)
  if(row.Recommendations  == 3 and row.compound > 0):
    hold_stocks.append(curr_stock)
  elif(row.Recommendations  <= 1.5 and row.compound > 0):
    buy_stocks.append(curr_stock)
  else:
    sell_stocks.append(curr_stock)

def recommend(stocks):
  for company, score in stocks:
    print(company +' - Public Opinion: ', end='')
    if(score > 0):
      print('GOOD')
    elif(0.5 >= score >= 0.5):
      print('NEUTRAL')
    else:
      print('BAD')
    print()

print('BUY STOCKS--------------------')
recommend(buy_stocks)

print('SELL STOCKS-------------------')
recommend(sell_stocks)

print('HOLD STOCKS-------------------')
recommend(hold_stocks)
  

BUY STOCKS--------------------
LKQ - Public Opinion: GOOD

PWR - Public Opinion: GOOD

SELL STOCKS-------------------
KSS - Public Opinion: BAD

UA - Public Opinion: BAD

HOLD STOCKS-------------------
ALLE - Public Opinion: GOOD

CLX - Public Opinion: GOOD

CTSH - Public Opinion: GOOD

DISCK - Public Opinion: GOOD

JKHY - Public Opinion: GOOD

KIM - Public Opinion: GOOD

L - Public Opinion: GOOD

LEG - Public Opinion: GOOD

ROK - Public Opinion: GOOD

SO - Public Opinion: GOOD

VTR - Public Opinion: GOOD

